In [1]:
import pandas as pd
import numpy as np
import wrangle as w
from datetime import datetime as dt

import seaborn as sns
import matplotlib.pyplot as plt
import time

In [ ]:
# We can use logistic regression to make a prediction (a probability between 0 and 1) of the home team winning or losing. 
# redo game_id with home team first then away team with og names  -- Done
# write down what encoded numbers mean for legend (i.e. 0 in OT means 'no' overtime, 1 'win" in home result) -- Done
# get stats to be away vs home stats instead of opp / team stats -- Done

In [124]:
dff = pd.read_csv('reordered_wip_df.csv')

In [8]:
nfl_df = pd.read_csv('nfl_data.csv')

In [21]:
df_2016 = nfl_df[(nfl_df['season'] >= 2015) & (nfl_df['season'] <= 2015)]

In [43]:
pd.options.display.max_rows = 20

In [44]:
df_2016[df_2016['Date'].str.contains('jan', case=False)]

,season,team,Week,Day,Date,Unnamed: 3,Unnamed: 4,OT,Unnamed: 6,Opp,...,FGA.1,XPM.1,XPA.1,Pnt.1,Yds.3.1,3DConv.1,3DAtt.1,4DConv.1,4DAtt.1,ToP.1
7663,2015,CRD,17,Sun,January 3,boxscore,L,NaN,NaN,Seattle Seahawks,...,4,3,4,2,91,8,15,1,1,36:37
7679,2015,ATL,17,Sun,January 3,boxscore,L,NaN,NaN,New Orleans Saints,...,2,2,2,2,105,6,10,0,1,29:34
7695,2015,RAV,17,Sun,January 3,boxscore,L,NaN,@,Cincinnati Bengals,...,1,3,3,7,303,0,9,1,1,30:44
7711,2015,BUF,17,Sun,January 3,boxscore,W,NaN,NaN,New York Jets,...,2,2,2,4,161,3,11,0,0,20:47
7727,2015,CAR,17,Sun,January 3,boxscore,W,NaN,NaN,Tampa Bay Buccaneers,...,1,1,1,4,147,8,16,1,2,28:10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8095,2015,SEA,17,Sun,January 3,boxscore,W,NaN,@,Arizona Cardinals,...,1,0,1,6,284,5,12,0,1,23:23
8111,2015,SFO,17,Sun,January 3,boxscore,W,OT,NaN,St. Louis Rams,...,5,1,1,5,259,3,13,0,1,38:30
8127,2015,TAM,17,Sun,January 3,boxscore,L,NaN,@,Carolina Panthers,...,1,5,5,2,76,7,10,0,0,31:50
8143,2015,OTI,17,Sun,January 3,boxscore,L,NaN,@,Indianapolis Colts,...,3,3,3,5,216,7,17,0,0,33:17


In [45]:
df[(df['parsed_date'] < pd.Timestamp('2015-02-01')) & (df['parsed_date'] > pd.Timestamp('2014-12-01'))]

,game_id,team_full_name,opp_full_name,home_team,away_team,team,parsed_date,season,month,week,...,xpm_opp,xpa_opp,pnt_opp,punt_yds_opp,3dconv_opp,3datt_opp,4dconv_opp,4datt_opp,opp_total_top,home_results
3580,2014_14_Arizona Cardinals_Kansas City Chiefs,Arizona Cardinals,Kansas City Chiefs,Arizona Cardinals,Kansas City Chiefs,CRD,2014-12-07,2014,12,14,...,2,2,4,197,6,14,0,2,1799,1
3581,2014_15_Arizona Cardinals_Los Angeles Rams,Arizona Cardinals,St. Louis Rams,Los Angeles Rams,Arizona Cardinals,CRD,2014-12-11,2014,12,15,...,0,0,8,404,4,15,0,1,1536,0
3582,2014_16_Arizona Cardinals_Seattle Seahawks,Arizona Cardinals,Seattle Seahawks,Arizona Cardinals,Seattle Seahawks,CRD,2014-12-21,2014,12,16,...,5,5,4,164,5,12,0,0,1983,0
3583,2014_17_Arizona Cardinals_San Francisco 49ers,Arizona Cardinals,San Francisco 49ers,San Francisco 49ers,Arizona Cardinals,CRD,2014-12-28,2014,12,17,...,2,2,4,196,5,12,0,0,1764,1
3595,2014_14_Atlanta Falcons_Green Bay Packers,Atlanta Falcons,Green Bay Packers,Green Bay Packers,Atlanta Falcons,ATL,2014-12-08,2014,12,14,...,4,5,1,31,7,12,0,1,1981,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4005,2015_17_Indianapolis Colts_Tennessee Titans,Indianapolis Colts,Tennessee Titans,Indianapolis Colts,Tennessee Titans,CLT,2015-01-03,2015,1,17,...,3,3,6,300,4,14,1,2,1603,1
4019,2015_17_Kansas City Chiefs_Las Vegas Raiders,Kansas City Chiefs,Oakland Raiders,Kansas City Chiefs,Las Vegas Raiders,KAN,2015-01-03,2015,1,17,...,2,2,7,271,3,14,1,1,1505,1
4032,2015_17_Los Angeles Rams_San Francisco 49ers,Los Angeles Rams,San Francisco 49ers,San Francisco 49ers,Los Angeles Rams,RAM,2015-01-03,2015,1,17,...,1,1,5,195,8,18,0,1,1983,1
4044,2015_17_Miami Dolphins_New England Patriots,Miami Dolphins,New England Patriots,Miami Dolphins,New England Patriots,MIA,2015-01-03,2015,1,17,...,1,1,7,361,4,14,0,1,1735,1


In [46]:
df[df['opp_full_name'] == 'St. Louis Rams']

    

,game_id,team_full_name,opp_full_name,home_team,away_team,team,parsed_date,season,month,week,...,xpm_opp,xpa_opp,pnt_opp,punt_yds_opp,3dconv_opp,3datt_opp,4dconv_opp,4datt_opp,opp_total_top,home_results
19,2000_4_Atlanta Falcons_Los Angeles Rams,Atlanta Falcons,St. Louis Rams,Atlanta Falcons,Los Angeles Rams,ATL,2000-09-24,2000,9,4,...,5,5,3,39,4,9,0,0,1636,0
22,2000_7_Atlanta Falcons_Los Angeles Rams,Atlanta Falcons,St. Louis Rams,Los Angeles Rams,Atlanta Falcons,ATL,2000-10-15,2000,10,7,...,1,1,3,32,7,14,2,3,2084,1
69,2000_10_Carolina Panthers_Los Angeles Rams,Carolina Panthers,St. Louis Rams,Los Angeles Rams,Carolina Panthers,CAR,2000-11-05,2000,11,10,...,3,3,2,124,7,14,1,1,1885,0
73,2000_14_Carolina Panthers_Los Angeles Rams,Carolina Panthers,St. Louis Rams,Carolina Panthers,Los Angeles Rams,CAR,2000-12-03,2000,12,14,...,0,0,1,34,6,12,0,2,1325,1
127,2000_1_Denver Broncos_Los Angeles Rams,Denver Broncos,St. Louis Rams,Los Angeles Rams,Denver Broncos,DEN,2000-09-04,2000,9,1,...,5,5,1,34,6,7,0,0,1667,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3908,2015_10_Chicago Bears_Los Angeles Rams,Chicago Bears,St. Louis Rams,Los Angeles Rams,Chicago Bears,CHI,2015-11-15,2015,11,10,...,1,1,6,296,4,14,0,2,1538,0
3923,2015_12_Cincinnati Bengals_Los Angeles Rams,Cincinnati Bengals,St. Louis Rams,Cincinnati Bengals,Los Angeles Rams,CIN,2015-11-29,2015,11,12,...,1,1,5,225,6,15,1,3,1793,1
3933,2015_7_Cleveland Browns_Los Angeles Rams,Cleveland Browns,St. Louis Rams,Los Angeles Rams,Cleveland Browns,CLE,2015-10-25,2015,10,7,...,3,3,5,244,1,9,0,0,1467,1
3973,2015_14_Detroit Lions_Los Angeles Rams,Detroit Lions,St. Louis Rams,Los Angeles Rams,Detroit Lions,DET,2015-12-13,2015,12,14,...,3,3,6,284,4,11,0,0,1570,1


In [83]:
before_2016 = df['season'] < 2016
before_2020 = df['season'] < 2020
before_2017 = df['season'] < 2017

before_2022 = ((df['season'] < 2022) & (df['season'] >= 2020))

# Step 2: Replace 'Los Angeles Rams' with 'St. Louis Rams' where the season is before 2016

In [85]:
df.loc[before_2016, 'home_team'] = df.loc[before_2016, 'home_team'].replace('Los Angeles Rams', 'St. Louis Rams')
df.loc[before_2016, 'away_team'] = df.loc[before_2016, 'away_team'].replace('Los Angeles Rams', 'St. Louis Rams')
df.loc[before_2020,'home_team'] = df.loc[before_2020, 'home_team'].replace ('Las Vegas Raiders', ' Oakland Raiders')
df.loc[before_2020,'away_team'] = df.loc[before_2020, 'away_team'].replace ('Las Vegas Raiders', ' Oakland Raiders')
df.loc[before_2020,'home_team'] = df.loc[before_2020, 'home_team'].replace ('Washington Commanders', ' Washington Redskins')
df.loc[before_2020,'away_team'] = df.loc[before_2020, 'away_team'].replace ('Washington Commanders', ' Washington Redskins')
df.loc[before_2022,'home_team'] = df.loc[before_2022, 'home_team'].replace ('Washington Commanders', ' Washington Football Team')
df.loc[before_2022,'away_team'] = df.loc[before_2022, 'away_team'].replace ('Washington Commanders', ' Washington Football Team')

In [96]:
team_changes =[
    ('Los Angeles Rams', 'St. Louis Rams', before_2016),
    ('Las Vegas Raiders', 'Oakland Raiders', before_2020),
    ('Washington Commanders', 'Washington Redskins', before_2020),
    ('Washington Commanders', 'Washington Football Team', before_2022),
    ('Los Angeles Chargers', 'San Diego Chargers', before_2017)
]

for old_name, new_name, condition in team_changes:
    for column in ['home_team', 'away_team']:
        df.loc[condition, column] = df.loc[condition, column].replace(old_name, new_name)

In [100]:
df[(df['opp_full_name'].str.contains('chargers', case = False)) & (df['season'] == 2022)]

,game_id,team_full_name,opp_full_name,home_team,away_team,team,parsed_date,season,month,week,...,xpm_opp,xpa_opp,pnt_opp,punt_yds_opp,3dconv_opp,3datt_opp,4dconv_opp,4datt_opp,opp_total_top,home_results
5643,2022_12_Arizona Cardinals_Los Angeles Chargers,Arizona Cardinals,Los Angeles Chargers,Arizona Cardinals,Los Angeles Chargers,CRD,2022-11-27,2022,11,12,...,2,2,6,234,6,13,0,0,1765,0
5657,2022_9_Atlanta Falcons_Los Angeles Chargers,Atlanta Falcons,Los Angeles Chargers,Atlanta Falcons,Los Angeles Chargers,ATL,2022-11-06,2022,11,9,...,2,2,4,168,8,16,1,1,1691,0
5738,2022_5_Cleveland Browns_Los Angeles Chargers,Cleveland Browns,Los Angeles Chargers,Cleveland Browns,Los Angeles Chargers,CLE,2022-10-09,2022,10,5,...,3,3,1,37,3,10,0,2,1798,0
5766,2022_6_Denver Broncos_Los Angeles Chargers,Denver Broncos,Los Angeles Chargers,Los Angeles Chargers,Denver Broncos,DEN,2022-10-17,2022,10,6,...,1,1,4,176,11,22,1,3,2329,1
5774,2022_18_Denver Broncos_Los Angeles Chargers,Denver Broncos,Los Angeles Chargers,Denver Broncos,Los Angeles Chargers,DEN,2022-01-08,2022,1,18,...,2,2,5,279,7,14,0,0,1737,1
5799,2022_4_Houston Texans_Los Angeles Chargers,Houston Texans,Los Angeles Chargers,Houston Texans,Los Angeles Chargers,HTX,2022-10-02,2022,10,4,...,4,4,4,185,8,16,1,2,1899,0
5822,2022_16_Indianapolis Colts_Los Angeles Chargers,Indianapolis Colts,Los Angeles Chargers,Indianapolis Colts,Los Angeles Chargers,CLT,2022-12-26,2022,12,16,...,2,2,4,153,8,18,1,1,2034,0
5825,2022_3_Jacksonville Jaguars_Los Angeles Chargers,Jacksonville Jaguars,Los Angeles Chargers,Los Angeles Chargers,Jacksonville Jaguars,JAX,2022-09-25,2022,9,3,...,1,1,4,175,5,13,0,2,1293,0
5833,2022_2_Kansas City Chiefs_Los Angeles Chargers,Kansas City Chiefs,Los Angeles Chargers,Kansas City Chiefs,Los Angeles Chargers,KAN,2022-09-15,2022,9,2,...,3,3,6,243,5,16,4,4,2017,1
5838,2022_11_Kansas City Chiefs_Los Angeles Chargers,Kansas City Chiefs,Los Angeles Chargers,Los Angeles Chargers,Kansas City Chiefs,KAN,2022-11-20,2022,11,11,...,3,3,3,147,8,14,0,0,1939,0


In [102]:
df.sample(10)

,game_id,team_full_name,opp_full_name,home_team,away_team,team,parsed_date,season,month,week,...,xpm_opp,xpa_opp,pnt_opp,punt_yds_opp,3dconv_opp,3datt_opp,4dconv_opp,4datt_opp,opp_total_top,home_results
1039,2004_1_Baltimore Ravens_Cleveland Browns,Baltimore Ravens,Cleveland Browns,Cleveland Browns,Baltimore Ravens,RAV,2004-09-12,2004,9,1,...,2,2,7,221,5,15,0,0,1741,1
5669,2022_5_Baltimore Ravens_Cincinnati Bengals,Baltimore Ravens,Cincinnati Bengals,Baltimore Ravens,Cincinnati Bengals,RAV,2022-10-09,2022,10,5,...,2,2,4,154,4,10,0,1,1801,1
2777,2010_6_New Orleans Saints_Tampa Bay Buccaneers,New Orleans Saints,Tampa Bay Buccaneers,Tampa Bay Buccaneers,New Orleans Saints,NOR,2010-10-17,2010,10,6,...,0,0,3,137,6,15,2,3,1711,0
2539,2009_13_San Francisco 49ers_Seattle Seahawks,Seattle Seahawks,San Francisco 49ers,Seattle Seahawks,San Francisco 49ers,SEA,2009-12-06,2009,12,13,...,2,2,9,446,1,13,1,2,1729,1
3640,2014_14_Carolina Panthers_New Orleans Saints,Carolina Panthers,New Orleans Saints,New Orleans Saints,Carolina Panthers,CAR,2014-12-07,2014,12,14,...,1,1,5,220,5,14,1,2,1440,0
501,2002_7_Arizona Cardinals_Dallas Cowboys,Arizona Cardinals,Dallas Cowboys,Arizona Cardinals,Dallas Cowboys,CRD,2002-10-20,2002,10,7,...,0,0,7,192,2,12,0,0,1857,1
2739,2010_17_Kansas City Chiefs_Las Vegas Raiders,Kansas City Chiefs,Oakland Raiders,Kansas City Chiefs,Oakland Raiders,KAN,2010-01-02,2010,1,17,...,4,4,5,234,6,13,0,1,2008,0
4623,2018_1_Baltimore Ravens_Buffalo Bills,Baltimore Ravens,Buffalo Bills,Baltimore Ravens,Buffalo Bills,RAV,2018-09-09,2018,9,1,...,0,0,8,397,2,15,0,1,1486,1
2611,2010_8_Carolina Panthers_Los Angeles Rams,Carolina Panthers,St. Louis Rams,St. Louis Rams,Carolina Panthers,CAR,2010-10-31,2010,10,8,...,2,2,5,241,6,15,0,0,2188,1
3905,2015_6_Chicago Bears_Detroit Lions,Chicago Bears,Detroit Lions,Detroit Lions,Chicago Bears,CHI,2015-10-18,2015,10,6,...,4,4,4,221,9,17,1,1,2337,1


In [105]:
df['game_id'] = df.apply(lambda row: f"{row['season']}_{row['week']}_{'_'.join([row['home_team'], row['away_team']])}", axis=1)

In [108]:
df.sample(7)

,game_id,team_full_name,opp_full_name,home_team,away_team,team,parsed_date,season,month,week,...,xpm_opp,xpa_opp,pnt_opp,punt_yds_opp,3dconv_opp,3datt_opp,4dconv_opp,4datt_opp,opp_total_top,home_results
5912,2023_10_Arizona Cardinals_Atlanta Falcons,Arizona Cardinals,Atlanta Falcons,Arizona Cardinals,Atlanta Falcons,CRD,2023-11-12,2023,11,10,...,2,2,4,203,7,14,1,2,1902,1
3375,2013_6_Minnesota Vikings_Carolina Panthers,Carolina Panthers,Minnesota Vikings,Minnesota Vikings,Carolina Panthers,CAR,2013-10-13,2013,10,6,...,1,1,4,199,6,14,1,2,1413,0
2744,2010_8_San Diego Chargers_Tennessee Titans,Los Angeles Chargers,Tennessee Titans,San Diego Chargers,Tennessee Titans,SDG,2010-10-31,2010,10,8,...,2,2,6,279,4,13,1,2,1333,1
318,2001_10_Carolina Panthers_San Francisco 49ers,Carolina Panthers,San Francisco 49ers,Carolina Panthers,San Francisco 49ers,CAR,2001-11-18,2001,11,10,...,2,2,7,240,10,20,1,1,2360,0
3998,2015_4_Indianapolis Colts_Jacksonville Jaguars,Indianapolis Colts,Jacksonville Jaguars,Indianapolis Colts,Jacksonville Jaguars,CLT,2015-10-04,2015,10,4,...,1,1,8,307,7,18,0,0,2170,1
3433,2013_13_Dallas Cowboys_ Oakland Raiders,Dallas Cowboys,Oakland Raiders,Dallas Cowboys,Oakland Raiders,DAL,2013-11-28,2013,11,13,...,3,3,5,265,7,13,0,0,1661,1
5602,2021_7_New England Patriots_New York Jets,New England Patriots,New York Jets,New England Patriots,New York Jets,NWE,2021-10-24,2021,10,7,...,1,1,2,100,7,13,0,2,1668,1


In [110]:
df.drop(columns = ['team_full_name', 'opp_full_name', 'team'], inplace=True)

In [112]:
df.head(3)

,game_id,home_team,away_team,parsed_date,season,month,week,day_of_week,pts,opp_pts,...,xpm_opp,xpa_opp,pnt_opp,punt_yds_opp,3dconv_opp,3datt_opp,4dconv_opp,4datt_opp,opp_total_top,home_results
0,2000_1_New York Giants_Arizona Cardinals,New York Giants,Arizona Cardinals,2000-09-03,2000,9,1,6,16,21,...,3,3,6,155,7,14,0,0,1890,1
1,2000_2_Arizona Cardinals_Dallas Cowboys,Arizona Cardinals,Dallas Cowboys,2000-09-10,2000,9,2,6,32,31,...,4,4,2,23,5,12,3,4,1720,1
2,2000_4_Arizona Cardinals_Green Bay Packers,Arizona Cardinals,Green Bay Packers,2000-09-24,2000,9,4,6,3,29,...,2,2,2,164,5,12,0,0,2188,0


In [83]:
team_mapping = {
    'Arizona Cardinals': 'Arizona Cardinals',
    'Atlanta Falcons': 'Atlanta Falcons',
    'Baltimore Ravens': 'Baltimore Ravens',
    'Buffalo Bills': 'Buffalo Bills',
    'Carolina Panthers': 'Carolina Panthers',
    'Chicago Bears': 'Chicago Bears',
    'Cincinnati Bengals': 'Cincinnati Bengals',
    'Cleveland Browns': 'Cleveland Browns',
    'Dallas Cowboys': 'Dallas Cowboys',
    'Denver Broncos': 'Denver Broncos',
    'Detroit Lions': 'Detroit Lions',
    'Green Bay Packers': 'Green Bay Packers',
    'Houston Texans': 'Houston Texans',
    'Indianapolis Colts': 'Indianapolis Colts',
    'Jacksonville Jaguars': 'Jacksonville Jaguars',
    'Kansas City Chiefs': 'Kansas City Chiefs',
    'Las Vegas Raiders': 'Las Vegas Raiders',  # Updated
    'Los Angeles Chargers': 'Los Angeles Chargers',  # Updated
    'Los Angeles Rams': 'Los Angeles Rams',  # Updated
    'Miami Dolphins': 'Miami Dolphins',
    'Minnesota Vikings': 'Minnesota Vikings',
    'New England Patriots': 'New England Patriots',
    'New Orleans Saints': 'New Orleans Saints',
    'New York Giants': 'New York Giants',
    'New York Jets': 'New York Jets',
    'Oakland Raiders': 'Las Vegas Raiders',  # Will be replaced
    'Philadelphia Eagles': 'Philadelphia Eagles',
    'Pittsburgh Steelers': 'Pittsburgh Steelers',
    'San Diego Chargers': 'Los Angeles Chargers',  # Will be replaced
    'San Francisco 49ers': 'San Francisco 49ers',
    'Seattle Seahawks': 'Seattle Seahawks',
    'St. Louis Rams': 'Los Angeles Rams',  # Will be replaced
    'Tampa Bay Buccaneers': 'Tampa Bay Buccaneers',
    'Tennessee Titans': 'Tennessee Titans',
    'Washington Commanders': 'Washington Commanders',  # Updated
    'Washington Football Team': 'Washington Commanders',  # Will be replaced
    'Washington Redskins': 'Washington Commanders'  # Will be replaced
}

In [119]:
df['home_team'] = df['home_team'].map(team_mapping)
df['away_team'] = df['away_team'].map(team_mapping)

In [120]:
df['away_team'].nunique()

32

In [123]:
df.head(3)

,game_id,home_team,away_team,parsed_date,season,month,week,day_of_week,pts,opp_pts,...,xpm_opp,xpa_opp,pnt_opp,punt_yds_opp,3dconv_opp,3datt_opp,4dconv_opp,4datt_opp,opp_total_top,home_results
0,2000_1_New York Giants_Arizona Cardinals,New York Giants,Arizona Cardinals,2000-09-03,2000,9,1,6,16,21,...,3,3,6,155,7,14,0,0,1890,1
1,2000_2_Arizona Cardinals_Dallas Cowboys,Arizona Cardinals,Dallas Cowboys,2000-09-10,2000,9,2,6,32,31,...,4,4,2,23,5,12,3,4,1720,1
2,2000_4_Arizona Cardinals_Green Bay Packers,Arizona Cardinals,Green Bay Packers,2000-09-24,2000,9,4,6,3,29,...,2,2,2,164,5,12,0,0,2188,0


In [127]:
cols_add_back = dff[['team_full_name', 'opp_full_name']]

In [129]:
df = pd.concat([df,cols_add_back], axis = 1)

In [130]:
df.head(3)

,game_id,home_team,away_team,parsed_date,season,month,week,day_of_week,pts,opp_pts,...,pnt_opp,punt_yds_opp,3dconv_opp,3datt_opp,4dconv_opp,4datt_opp,opp_total_top,home_results,team_full_name,opp_full_name
0,2000_1_New York Giants_Arizona Cardinals,New York Giants,Arizona Cardinals,2000-09-03,2000,9,1,6,16,21,...,6,155,7,14,0,0,1890,1,Arizona Cardinals,New York Giants
1,2000_2_Arizona Cardinals_Dallas Cowboys,Arizona Cardinals,Dallas Cowboys,2000-09-10,2000,9,2,6,32,31,...,2,23,5,12,3,4,1720,1,Arizona Cardinals,Dallas Cowboys
2,2000_4_Arizona Cardinals_Green Bay Packers,Arizona Cardinals,Green Bay Packers,2000-09-24,2000,9,4,6,3,29,...,2,164,5,12,0,0,2188,0,Arizona Cardinals,Green Bay Packers


In [140]:
df['team_full_name'] = df['team_full_name'].map(team_mapping)
df['opp_full_name'] = df['opp_full_name'].map(team_mapping)

In [141]:
df['opp_full_name'].nunique()

31

In [147]:
# Step 1: Determine if home team is team_full_name or opponent
df['home_is_team'] = df['home_team'] == df['team_full_name']

In [148]:
df

,game_id,home_team,away_team,parsed_date,season,month,week,day_of_week,pts,opp_pts,...,punt_yds_opp,3dconv_opp,3datt_opp,4dconv_opp,4datt_opp,opp_total_top,home_results,team_full_name,opp_full_name,home_is_team
0,2000_1_New York Giants_Arizona Cardinals,New York Giants,Arizona Cardinals,2000-09-03,2000,9,1,6,16,21,...,155,7,14,0,0,1890,1,Arizona Cardinals,New York Giants,False
1,2000_2_Arizona Cardinals_Dallas Cowboys,Arizona Cardinals,Dallas Cowboys,2000-09-10,2000,9,2,6,32,31,...,23,5,12,3,4,1720,1,Arizona Cardinals,Dallas Cowboys,True
2,2000_4_Arizona Cardinals_Green Bay Packers,Arizona Cardinals,Green Bay Packers,2000-09-24,2000,9,4,6,3,29,...,164,5,12,0,0,2188,0,Arizona Cardinals,Green Bay Packers,True
3,2000_5_San Francisco 49ers_Arizona Cardinals,San Francisco 49ers,Arizona Cardinals,2000-10-01,2000,10,5,6,20,27,...,137,7,16,0,0,1930,1,Arizona Cardinals,San Francisco 49ers,False
4,2000_6_Arizona Cardinals_Cleveland Browns,Arizona Cardinals,Cleveland Browns,2000-10-08,2000,10,6,6,29,21,...,206,5,14,0,1,1737,1,Arizona Cardinals,Cleveland Browns,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6170,2023_14_San Francisco 49ers_Seattle Seahawks,San Francisco 49ers,Seattle Seahawks,2023-12-10,2023,12,14,6,16,28,...,246,6,11,0,0,1755,1,Seattle Seahawks,San Francisco 49ers,False
6171,2023_16_Tennessee Titans_Seattle Seahawks,Tennessee Titans,Seattle Seahawks,2023-12-24,2023,12,16,6,20,17,...,117,6,12,1,1,2031,0,Seattle Seahawks,Tennessee Titans,False
6172,2023_11_San Francisco 49ers_Tampa Bay Buccaneers,San Francisco 49ers,Tampa Bay Buccaneers,2023-11-19,2023,11,11,6,27,14,...,144,7,14,1,3,1749,1,San Francisco 49ers,Tampa Bay Buccaneers,True
6173,2023_17_Washington Commanders_San Francisco 49ers,Washington Commanders,San Francisco 49ers,2023-12-31,2023,12,17,6,27,10,...,156,3,9,0,1,1307,0,San Francisco 49ers,Washington Commanders,False


In [151]:
df.columns

Index(['game_id', 'home_team', 'away_team', 'parsed_date', 'season', 'month',
       'week', 'day_of_week', 'pts', 'opp_pts', 'ot', 'pass_cmp', 'pass_att',
       'pass_yds', 'pass_td', 'int', 'sk', 'yds_lost_sks', 'yds_pass_att',
       'net_yds_pass_att', 'cmp%', 'pass_rating', 'rush_att', 'rush_yds',
       'rush_yds_att', 'rush_td', 'fgm', 'fga', 'xpm', 'xpa', 'pnt',
       'punt_yds', '3dconv', '3datt', '4dconv', '4datt', 'total_top',
       'pass_cmp_opp', 'pass_att_opp', 'pass_yds_opp', 'pass_td_opp',
       'int_opp', 'sk_opp', 'yds_lost_sks_opp', 'yds_pass_att_opp',
       'net_yds_pass_att_opp', 'cmp%_opp', 'pass_rating_opp', 'rush_att_opp',
       'rush_yds_opp', 'rush_yds_att_opp', 'rush_td_opp', 'fgm_opp', 'fga_opp',
       'xpm_opp', 'xpa_opp', 'pnt_opp', 'punt_yds_opp', '3dconv_opp',
       '3datt_opp', '4dconv_opp', '4datt_opp', 'opp_total_top', 'home_results',
       'team_full_name', 'opp_full_name', 'home_is_team'],
      dtype='object')

In [152]:
# Step 2 & 3: Create and assign statistics for home and away teams
stats_columns = ['pts', 'pass_cmp', 'pass_att', 'pass_yds', 'pass_td', 'int', 'sk',
                 'yds_lost_sks', 'yds_pass_att','net_yds_pass_att', 'cmp%', 'pass_rating', 'rush_att',
                 'rush_yds','rush_yds_att', 'rush_td', 'fgm', 'fga', 'xpm', 'xpa', 'pnt','punt_yds', '3dconv',
                 '3datt', '4dconv', '4datt', 'total_top']
opp_stats_columns = ['opp_pts','pass_cmp_opp', 'pass_att_opp', 'pass_yds_opp', 'pass_td_opp','int_opp', 'sk_opp',
                     'yds_lost_sks_opp', 'yds_pass_att_opp','net_yds_pass_att_opp', 'cmp%_opp', 'pass_rating_opp',
                     'rush_att_opp','rush_yds_opp', 'rush_yds_att_opp', 'rush_td_opp', 'fgm_opp', 'fga_opp',
                     'xpm_opp', 'xpa_opp', 'pnt_opp', 'punt_yds_opp', '3dconv_opp','3datt_opp', '4dconv_opp',
                     '4datt_opp', 'opp_total_top']

for team_stat, opp_stat in zip(stats_columns, opp_stats_columns):
    df[f'home_{team_stat}'] = df.apply(lambda row: row[team_stat] if row['home_is_team'] else row[opp_stat], axis=1)
    df[f'away_{team_stat}'] = df.apply(lambda row: row[opp_stat] if row['home_is_team'] else row[team_stat], axis=1)

In [153]:
df

,game_id,home_team,away_team,parsed_date,season,month,week,day_of_week,pts,opp_pts,...,home_3dconv,away_3dconv,home_3datt,away_3datt,home_4dconv,away_4dconv,home_4datt,away_4datt,home_total_top,away_total_top
0,2000_1_New York Giants_Arizona Cardinals,New York Giants,Arizona Cardinals,2000-09-03,2000,9,1,6,16,21,...,7,6,14,15,0,0,0,2,1890,1710
1,2000_2_Arizona Cardinals_Dallas Cowboys,Arizona Cardinals,Dallas Cowboys,2000-09-10,2000,9,2,6,32,31,...,7,5,12,12,0,3,0,4,1880,1720
2,2000_4_Arizona Cardinals_Green Bay Packers,Arizona Cardinals,Green Bay Packers,2000-09-24,2000,9,4,6,3,29,...,6,5,14,12,0,0,1,0,1412,2188
3,2000_5_San Francisco 49ers_Arizona Cardinals,San Francisco 49ers,Arizona Cardinals,2000-10-01,2000,10,5,6,20,27,...,7,3,16,13,0,2,0,3,1930,1670
4,2000_6_Arizona Cardinals_Cleveland Browns,Arizona Cardinals,Cleveland Browns,2000-10-08,2000,10,6,6,29,21,...,2,5,10,14,1,0,2,1,1863,1737
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6170,2023_14_San Francisco 49ers_Seattle Seahawks,San Francisco 49ers,Seattle Seahawks,2023-12-10,2023,12,14,6,16,28,...,6,2,11,11,0,0,0,1,1755,1845
6171,2023_16_Tennessee Titans_Seattle Seahawks,Tennessee Titans,Seattle Seahawks,2023-12-24,2023,12,16,6,20,17,...,6,7,12,13,1,0,1,0,2031,1569
6172,2023_11_San Francisco 49ers_Tampa Bay Buccaneers,San Francisco 49ers,Tampa Bay Buccaneers,2023-11-19,2023,11,11,6,27,14,...,4,7,10,14,0,1,1,3,1851,1749
6173,2023_17_Washington Commanders_San Francisco 49ers,Washington Commanders,San Francisco 49ers,2023-12-31,2023,12,17,6,27,10,...,3,4,9,9,0,0,1,1,1307,2293


In [154]:
# Optionally, drop the original and opponent stat columns
df = df.drop(columns=stats_columns + opp_stats_columns)

In [159]:
pd.options.display.max_columns= None 
df.drop(columns=['team_full_name','opp_full_name','home_is_team'], inplace=True)

In [11]:
df['home_results'] = df.pop('home_results')

In [164]:
df.to_csv('getting_there.csv', index = False)

In [174]:
df.rename(columns = {'parsed_date': 'date', 'ot':'overtime','home_total_top': 'home_top',
                     'away_total_top': 'away_top'},inplace = True)

In [168]:
df = df.sort_values(by='parsed_date', ascending=True,ignore_index=True)

In [176]:
df.to_csv('getting_there.csv', index = False)

In [ ]:
#rename columns -- done --
#reorder according to date -- done --

In [178]:
# load with this code snippet for dt object 
# pd.read_csv('getting_there.csv', parse_dates=['date'], date_format='%Y-%m-%d')

In [13]:
df = pd.read_csv('getting_there.csv', parse_dates=['date'], date_format='%Y-%m-%d')
df.head()

,game_id,home_team,away_team,date,season,month,week,day_of_week,overtime,home_pts,...,away_3dconv,home_3datt,away_3datt,home_4dconv,away_4dconv,home_4datt,away_4datt,home_top,away_top,home_results
0,2000_1_New York Giants_Arizona Cardinals,New York Giants,Arizona Cardinals,2000-09-03,2000,9,1,6,0,21,...,6,14,15,0,0,0,2,1890,1710,1
1,2000_1_Minnesota Vikings_Chicago Bears,Minnesota Vikings,Chicago Bears,2000-09-03,2000,9,1,6,0,30,...,9,13,15,0,0,0,0,1646,1954,1
2,2000_1_Cleveland Browns_Jacksonville Jaguars,Cleveland Browns,Jacksonville Jaguars,2000-09-03,2000,9,1,6,0,7,...,10,10,16,1,0,2,0,1377,2223,0
3,2000_1_Dallas Cowboys_Philadelphia Eagles,Dallas Cowboys,Philadelphia Eagles,2000-09-03,2000,9,1,6,0,14,...,7,13,12,0,0,0,0,1230,2370,0
4,2000_1_Buffalo Bills_Tennessee Titans,Buffalo Bills,Tennessee Titans,2000-09-03,2000,9,1,6,0,16,...,2,16,14,0,0,0,1,1991,1609,1


In [14]:
# Point difference
df['pts_diff_home'] = df['home_pts'] - df['away_pts']

# Passing yards difference
df['pass_yds_diff_home'] = df['home_pass_yds'] - df['away_pass_yds']

# Rushing yards difference
df['rush_yds_diff_home'] = df['home_rush_yds'] - df['away_rush_yds']

# Time of possession difference
df['top_diff_home'] = df['home_top'] - df['away_top']

In [16]:
df['home_results'] = df.pop('home_results')

In [17]:
df

,game_id,home_team,away_team,date,season,month,week,day_of_week,overtime,home_pts,...,away_4dconv,home_4datt,away_4datt,home_top,away_top,pts_diff,pass_yds_diff,rush_yds_diff,top_diff,home_results
0,2000_1_New York Giants_Arizona Cardinals,New York Giants,Arizona Cardinals,2000-09-03,2000,9,1,6,0,21,...,0,0,2,1890,1710,5,-140,180,180,1
1,2000_1_Minnesota Vikings_Chicago Bears,Minnesota Vikings,Chicago Bears,2000-09-03,2000,9,1,6,0,30,...,0,0,0,1646,1954,3,-84,33,-308,1
2,2000_1_Cleveland Browns_Jacksonville Jaguars,Cleveland Browns,Jacksonville Jaguars,2000-09-03,2000,9,1,6,0,7,...,0,2,0,1377,2223,-20,-126,-23,-846,0
3,2000_1_Dallas Cowboys_Philadelphia Eagles,Dallas Cowboys,Philadelphia Eagles,2000-09-03,2000,9,1,6,0,14,...,0,0,0,1230,2370,-27,-19,-239,-1140,0
4,2000_1_Buffalo Bills_Tennessee Titans,Buffalo Bills,Tennessee Titans,2000-09-03,2000,9,1,6,0,16,...,0,0,1,1991,1609,3,18,76,382,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6170,2023_17_Chicago Bears_Atlanta Falcons,Chicago Bears,Atlanta Falcons,2023-12-31,2023,12,17,6,0,37,...,0,2,1,2234,1366,20,67,58,868,1
6171,2023_17_Houston Texans_Tennessee Titans,Houston Texans,Tennessee Titans,2023-12-31,2023,12,17,6,0,26,...,2,0,4,1991,1609,23,67,58,382,1
6172,2023_17_Baltimore Ravens_Miami Dolphins,Baltimore Ravens,Miami Dolphins,2023-12-31,2023,12,17,6,0,56,...,2,1,4,1823,1777,37,110,6,46,1
6173,2023_17_Denver Broncos_Los Angeles Chargers,Denver Broncos,Los Angeles Chargers,2023-12-31,2023,12,17,6,0,16,...,1,1,2,2026,1574,7,-3,15,452,1


In [24]:
df['season'].value_counts().sort_index(ascending=True)

season
2000    248
2001    248
2002    256
2003    256
2004    256
2005    256
2006    256
2007    256
2008    256
2009    256
2010    256
2011    256
2012    256
2013    256
2014    256
2015    256
2016    256
2017    256
2018    256
2019    256
2020    256
2021    272
2022    271
2023    272
Name: count, dtype: int64

In [31]:
df[(df['home_team'] == 'Los Angeles Chargers') & (df['season'] == 2023)].sort_values(by = 'week')

,game_id,home_team,away_team,date,season,month,week,day_of_week,overtime,home_pts,...,away_4dconv,home_4datt,away_4datt,home_top,away_top,pts_diff,pass_yds_diff,rush_yds_diff,top_diff,home_results
5930,2023_1_Los Angeles Chargers_Miami Dolphins,Los Angeles Chargers,Miami Dolphins,2023-09-10,2023,9,1,6,0,34,...,1,1,1,1961,1639,-2,-266,163,322,0
5968,2023_4_Los Angeles Chargers_Las Vegas Raiders,Los Angeles Chargers,Las Vegas Raiders,2023-10-01,2023,10,4,6,0,24,...,3,2,3,1629,1971,7,-38,79,-342,1
6011,2023_6_Los Angeles Chargers_Dallas Cowboys,Los Angeles Chargers,Dallas Cowboys,2023-10-16,2023,10,6,0,0,17,...,1,2,2,1808,1792,-3,-27,-43,16,0
6034,2023_8_Los Angeles Chargers_Chicago Bears,Los Angeles Chargers,Chicago Bears,2023-10-29,2023,10,8,6,0,30,...,1,1,4,1888,1712,17,76,-19,176,1
6065,2023_10_Los Angeles Chargers_Detroit Lions,Los Angeles Chargers,Detroit Lions,2023-11-12,2023,11,10,6,0,38,...,4,3,5,1766,1834,-3,-10,-102,-68,0
6093,2023_12_Los Angeles Chargers_Baltimore Ravens,Los Angeles Chargers,Baltimore Ravens,2023-11-26,2023,11,12,6,0,10,...,1,3,3,1597,2003,-10,29,-111,-406,0
6117,2023_14_Los Angeles Chargers_Denver Broncos,Los Angeles Chargers,Denver Broncos,2023-12-10,2023,12,14,6,0,7,...,0,6,0,1591,2009,-17,-9,-30,-418,0
6144,2023_16_Los Angeles Chargers_Buffalo Bills,Los Angeles Chargers,Buffalo Bills,2023-12-23,2023,12,16,5,0,22,...,0,0,0,1984,1616,-2,-55,-7,368,0
5908,2023_18_Los Angeles Chargers_Kansas City Chiefs,Los Angeles Chargers,Kansas City Chiefs,2023-01-07,2023,1,18,5,0,12,...,1,3,1,1975,1625,-1,90,-5,350,0


In [40]:
jan_games = df['date'].dt.month == 1

In [41]:
jan_games

0       False
1       False
2       False
3       False
4       False
        ...  
6170    False
6171    False
6172    False
6173    False
6174    False
Name: date, Length: 6175, dtype: bool

In [42]:
# Add one year to the date for January games
df.loc[jan_games, 'date'] = df.loc[jan_games, 'date'] + pd.DateOffset(years=1)

In [46]:
df[jan_games].head()

,game_id,home_team,away_team,date,season,month,week,day_of_week,overtime,home_pts,...,away_4dconv,home_4datt,away_4datt,home_top,away_top,pts_diff,pass_yds_diff,rush_yds_diff,top_diff,home_results
248,2001_17_Miami Dolphins_Buffalo Bills,Miami Dolphins,Buffalo Bills,2002-01-06,2001,1,17,5,0,34,...,1,0,2,2144,1456,27,-124,163,688,1
249,2001_17_Carolina Panthers_New England Patriots,Carolina Panthers,New England Patriots,2002-01-06,2001,1,17,5,0,6,...,1,2,2,1599,2001,-32,-67,91,-402,0
250,2001_17_Chicago Bears_Jacksonville Jaguars,Chicago Bears,Jacksonville Jaguars,2002-01-06,2001,1,17,5,0,33,...,0,2,0,2113,1487,20,7,78,626,1
251,2001_17_Tennessee Titans_Cincinnati Bengals,Tennessee Titans,Cincinnati Bengals,2002-01-06,2001,1,17,5,0,21,...,1,1,2,1730,1870,-2,-64,-43,-140,0
252,2001_17_Pittsburgh Steelers_Cleveland Browns,Pittsburgh Steelers,Cleveland Browns,2002-01-06,2001,1,17,5,0,28,...,0,0,1,2084,1516,21,50,153,568,1


In [47]:
df.sort_values(by = 'date', inplace=True)

In [51]:
df.reset_index(drop= 'index',inplace=True)

In [54]:
df.drop(columns= 'index', inplace= True)

In [56]:
df.rename(columns={'pts_diff':'pts_diff_home', 'pass_yds_diff':'pass_yds_diff_home',
                   'rush_yds_diff':'rush_yds_diff_home', 'top_diff':'top_diff_home'},inplace=True)	

In [57]:
df

,game_id,home_team,away_team,date,season,month,week,day_of_week,overtime,home_pts,...,away_4dconv,home_4datt,away_4datt,home_top,away_top,pts_diff_home,pass_yds_diff_home,rush_yds_diff_home,top_diff_home,home_results
0,2000_1_New York Giants_Arizona Cardinals,New York Giants,Arizona Cardinals,2000-09-03,2000,9,1,6,0,21,...,0,0,2,1890,1710,5,-140,180,180,1
1,2000_1_New England Patriots_Tampa Bay Buccaneers,New England Patriots,Tampa Bay Buccaneers,2000-09-03,2000,9,1,6,0,16,...,0,2,0,1705,1895,-5,34,-52,-190,0
2,2000_1_Pittsburgh Steelers_Baltimore Ravens,Pittsburgh Steelers,Baltimore Ravens,2000-09-03,2000,9,1,6,0,0,...,1,2,1,1493,2107,-16,-3,-110,-614,0
3,2000_1_Atlanta Falcons_San Francisco 49ers,Atlanta Falcons,San Francisco 49ers,2000-09-03,2000,9,1,6,0,36,...,0,0,1,1899,1701,8,17,3,198,1
4,2000_1_ Oakland Raiders_San Diego Chargers,NaN,Los Angeles Chargers,2000-09-03,2000,9,1,6,0,9,...,0,0,1,1890,1710,3,-13,-9,180,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6170,2023_18_Washington Commanders_Dallas Cowboys,Washington Commanders,Dallas Cowboys,2024-01-07,2023,1,18,5,0,10,...,1,4,1,1565,2035,-28,-179,-81,-470,0
6171,2023_18_Cincinnati Bengals_Cleveland Browns,Cincinnati Bengals,Cleveland Browns,2024-01-07,2023,1,18,5,0,31,...,1,0,1,2146,1454,17,5,79,692,1
6172,2023_18_Tennessee Titans_Jacksonville Jaguars,Tennessee Titans,Jacksonville Jaguars,2024-01-07,2023,1,18,5,0,28,...,0,1,3,1663,1937,8,-128,93,-274,1
6173,2023_18_Carolina Panthers_Tampa Bay Buccaneers,Carolina Panthers,Tampa Bay Buccaneers,2024-01-07,2023,1,18,5,0,0,...,0,2,0,1716,1884,-9,-47,18,-168,0


In [58]:
df.to_csv('getting_there.csv', index = False)

In [80]:
df.isna().sum().sort_values()

game_id              0
away_rush_yds        0
home_rush_yds_att    0
away_rush_yds_att    0
home_rush_td         0
                    ..
away_int             0
home_int             0
away_pass_td         0
away_yds_lost_sks    0
home_results         0
Length: 68, dtype: int64

In [118]:
def extract_teams(game_id):
    parts = game_id.split('_')
    if len(parts) >= 4:
        home_team = ''.join(parts[2:-1])
        away_team = parts[-1]
        return home_team, away_team
    return None, None

In [119]:
# Create a mask for rows where either home_team or away_team is NaN
mask = df['home_team'].isna() | df['away_team'].isna()

# Apply the function to extract team names
df.loc[mask, ['home_team', 'away_team']] = df.loc[mask, 'game_id'].apply(extract_teams).tolist()

In [120]:
df['home_team'] = df['home_team'].map(team_mapping)
df['away_team'] = df['away_team'].map(team_mapping)

In [125]:
df.isna().sum()

game_id               0
home_team             0
away_team             0
date                  0
season                0
                     ..
pts_diff_home         0
pass_yds_diff_home    0
rush_yds_diff_home    0
top_diff_home         0
home_results          0
Length: 68, dtype: int64

In [122]:
df['home_team'].sort_values().unique().tolist()

['Arizona Cardinals',
 'Atlanta Falcons',
 'Baltimore Ravens',
 'Buffalo Bills',
 'Carolina Panthers',
 'Chicago Bears',
 'Cincinnati Bengals',
 'Cleveland Browns',
 'Dallas Cowboys',
 'Denver Broncos',
 'Detroit Lions',
 'Green Bay Packers',
 'Houston Texans',
 'Indianapolis Colts',
 'Jacksonville Jaguars',
 'Kansas City Chiefs',
 'Las Vegas Raiders',
 'Los Angeles Chargers',
 'Los Angeles Rams',
 'Miami Dolphins',
 'Minnesota Vikings',
 'New England Patriots',
 'New Orleans Saints',
 'New York Giants',
 'New York Jets',
 'Philadelphia Eagles',
 'Pittsburgh Steelers',
 'San Francisco 49ers',
 'Seattle Seahawks',
 'Tampa Bay Buccaneers',
 'Tennessee Titans',
 'Washington Commanders']

In [114]:
def clean_game_id(game_id):
    # Split the game_id into parts
    parts = game_id.split('_')
    # Clean up any leading/trailing spaces in each part
    parts = [part.strip() for part in parts]
    # Rejoin the parts with underscores
    return '_'.join(parts)

df['game_id'] = df['game_id'].apply(clean_game_id)

In [126]:
df.to_csv('getting_there.csv', index = False)

In [127]:
df

,game_id,home_team,away_team,date,season,month,week,day_of_week,overtime,home_pts,...,away_4dconv,home_4datt,away_4datt,home_top,away_top,pts_diff_home,pass_yds_diff_home,rush_yds_diff_home,top_diff_home,home_results
0,2000_1_New York Giants_Arizona Cardinals,New York Giants,Arizona Cardinals,2000-09-03,2000,9,1,6,0,21,...,0,0,2,1890,1710,5,-140,180,180,1
1,2000_1_New England Patriots_Tampa Bay Buccaneers,New England Patriots,Tampa Bay Buccaneers,2000-09-03,2000,9,1,6,0,16,...,0,2,0,1705,1895,-5,34,-52,-190,0
2,2000_1_Pittsburgh Steelers_Baltimore Ravens,Pittsburgh Steelers,Baltimore Ravens,2000-09-03,2000,9,1,6,0,0,...,1,2,1,1493,2107,-16,-3,-110,-614,0
3,2000_1_Atlanta Falcons_San Francisco 49ers,Atlanta Falcons,San Francisco 49ers,2000-09-03,2000,9,1,6,0,36,...,0,0,1,1899,1701,8,17,3,198,1
4,2000_1_Oakland Raiders_San Diego Chargers,Las Vegas Raiders,Los Angeles Chargers,2000-09-03,2000,9,1,6,0,9,...,0,0,1,1890,1710,3,-13,-9,180,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6170,2023_18_Washington Commanders_Dallas Cowboys,Washington Commanders,Dallas Cowboys,2024-01-07,2023,1,18,5,0,10,...,1,4,1,1565,2035,-28,-179,-81,-470,0
6171,2023_18_Cincinnati Bengals_Cleveland Browns,Cincinnati Bengals,Cleveland Browns,2024-01-07,2023,1,18,5,0,31,...,1,0,1,2146,1454,17,5,79,692,1
6172,2023_18_Tennessee Titans_Jacksonville Jaguars,Tennessee Titans,Jacksonville Jaguars,2024-01-07,2023,1,18,5,0,28,...,0,1,3,1663,1937,8,-128,93,-274,1
6173,2023_18_Carolina Panthers_Tampa Bay Buccaneers,Carolina Panthers,Tampa Bay Buccaneers,2024-01-07,2023,1,18,5,0,0,...,0,2,0,1716,1884,-9,-47,18,-168,0


In [2]:
# write down what encoded numbers mean for legend (i.e. 0 in OT means 'no' overtime, 1 'win" in home result) -- DONE
# research if i need to encode categorical features
# should i make avg columns of teams as they go on during the season?
# pd.read_csv('getting_there.csv', parse_dates=['date'], date_format='%Y-%m-%d')

In [33]:
df = pd.read_csv('getting_there.csv', parse_dates=['date'], date_format='%Y-%m-%d')
df.head()

,game_id,home_team,away_team,date,season,month,week,day_of_week,overtime,home_pts,away_pts,home_pass_cmp,away_pass_cmp,home_pass_att,away_pass_att,home_pass_yds,away_pass_yds,home_pass_td,away_pass_td,home_int,away_int,home_sk,away_sk,home_yds_lost_sks,away_yds_lost_sks,home_yds_pass_att,away_yds_pass_att,home_net_yds_pass_att,away_net_yds_pass_att,home_cmp%,away_cmp%,home_pass_rating,away_pass_rating,home_rush_att,away_rush_att,home_rush_yds,away_rush_yds,home_rush_yds_att,away_rush_yds_att,home_rush_td,away_rush_td,home_fgm,away_fgm,home_fga,away_fga,home_xpm,away_xpm,home_xpa,away_xpa,home_pnt,away_pnt,home_punt_yds,away_punt_yds,home_3dconv,away_3dconv,home_3datt,away_3datt,home_4dconv,away_4dconv,home_4datt,away_4datt,home_top,away_top,pts_diff_home,pass_yds_diff_home,rush_yds_diff_home,top_diff_home,home_results
0,2000_1_New York Giants_Arizona Cardinals,New York Giants,Arizona Cardinals,2000-09-03,2000,9,1,6,0,21,16,17,28,25,49,172,312,0,2,1,3,0,1,0,6,6.9,6.5,6.9,6.2,68.0,57.1,70.7,64.3,41,20,223,43,5.4,2.2,3,0,0,1,0,1,3,1,3,1,6,5,155,210,7,6,14,15,0,0,0,2,1890,1710,5,-140,180,180,1
1,2000_1_New England Patriots_Tampa Bay Buccaneers,New England Patriots,Tampa Bay Buccaneers,2000-09-03,2000,9,1,6,0,16,21,26,12,39,24,190,156,1,1,0,0,6,1,26,11,5.5,7.0,4.2,6.2,66.7,50.0,86.5,84.7,21,38,88,140,4.2,3.7,0,2,1,0,2,0,1,3,1,3,9,10,369,400,4,6,17,16,1,0,2,0,1705,1895,-5,34,-52,-190,0
2,2000_1_Pittsburgh Steelers_Baltimore Ravens,Pittsburgh Steelers,Baltimore Ravens,2000-09-03,2000,9,1,6,0,0,16,17,18,39,32,193,196,0,1,0,0,1,1,6,3,5.1,6.2,4.8,5.9,43.6,56.3,59.0,84.9,18,36,30,140,1.7,3.9,0,0,0,3,1,3,0,1,0,1,6,7,184,280,4,5,14,16,0,1,2,1,1493,2107,-16,-3,-110,-614,0
3,2000_1_Atlanta Falcons_San Francisco 49ers,Atlanta Falcons,San Francisco 49ers,2000-09-03,2000,9,1,6,0,36,28,16,23,31,36,264,247,2,3,0,1,0,1,0,6,8.5,7.0,8.5,6.7,51.6,63.9,102.1,100.1,32,24,95,92,3.0,3.8,0,1,5,0,5,0,3,4,3,4,2,5,96,98,4,2,11,9,0,0,0,1,1899,1701,8,17,3,198,1
4,2000_1_Oakland Raiders_San Diego Chargers,Las Vegas Raiders,Los Angeles Chargers,2000-09-03,2000,9,1,6,0,9,6,20,19,35,42,166,179,1,0,0,3,4,2,10,18,5.0,4.7,4.3,4.1,57.1,45.2,79.0,27.8,25,28,67,76,2.7,2.7,0,1,0,0,1,0,1,0,1,0,10,7,420,343,3,8,16,18,0,0,0,1,1890,1710,3,-13,-9,180,1


In [11]:
pd.options.display.max_columns = None
df.head()

,game_id,home_team,away_team,date,season,month,week,day_of_week,overtime,home_pts,away_pts,home_pass_cmp,away_pass_cmp,home_pass_att,away_pass_att,home_pass_yds,away_pass_yds,home_pass_td,away_pass_td,home_int,away_int,home_sk,away_sk,home_yds_lost_sks,away_yds_lost_sks,home_yds_pass_att,away_yds_pass_att,home_net_yds_pass_att,away_net_yds_pass_att,home_cmp%,away_cmp%,home_pass_rating,away_pass_rating,home_rush_att,away_rush_att,home_rush_yds,away_rush_yds,home_rush_yds_att,away_rush_yds_att,home_rush_td,away_rush_td,home_fgm,away_fgm,home_fga,away_fga,home_xpm,away_xpm,home_xpa,away_xpa,home_pnt,away_pnt,home_punt_yds,away_punt_yds,home_3dconv,away_3dconv,home_3datt,away_3datt,home_4dconv,away_4dconv,home_4datt,away_4datt,home_top,away_top,pts_diff_home,pass_yds_diff_home,rush_yds_diff_home,top_diff_home,home_results
0,2000_1_New York Giants_Arizona Cardinals,New York Giants,Arizona Cardinals,2000-09-03,2000,9,1,6,0,21,16,17,28,25,49,172,312,0,2,1,3,0,1,0,6,6.9,6.5,6.9,6.2,68.0,57.1,70.7,64.3,41,20,223,43,5.4,2.2,3,0,0,1,0,1,3,1,3,1,6,5,155,210,7,6,14,15,0,0,0,2,1890,1710,5,-140,180,180,1
1,2000_1_New England Patriots_Tampa Bay Buccaneers,New England Patriots,Tampa Bay Buccaneers,2000-09-03,2000,9,1,6,0,16,21,26,12,39,24,190,156,1,1,0,0,6,1,26,11,5.5,7.0,4.2,6.2,66.7,50.0,86.5,84.7,21,38,88,140,4.2,3.7,0,2,1,0,2,0,1,3,1,3,9,10,369,400,4,6,17,16,1,0,2,0,1705,1895,-5,34,-52,-190,0
2,2000_1_Pittsburgh Steelers_Baltimore Ravens,Pittsburgh Steelers,Baltimore Ravens,2000-09-03,2000,9,1,6,0,0,16,17,18,39,32,193,196,0,1,0,0,1,1,6,3,5.1,6.2,4.8,5.9,43.6,56.3,59.0,84.9,18,36,30,140,1.7,3.9,0,0,0,3,1,3,0,1,0,1,6,7,184,280,4,5,14,16,0,1,2,1,1493,2107,-16,-3,-110,-614,0
3,2000_1_Atlanta Falcons_San Francisco 49ers,Atlanta Falcons,San Francisco 49ers,2000-09-03,2000,9,1,6,0,36,28,16,23,31,36,264,247,2,3,0,1,0,1,0,6,8.5,7.0,8.5,6.7,51.6,63.9,102.1,100.1,32,24,95,92,3.0,3.8,0,1,5,0,5,0,3,4,3,4,2,5,96,98,4,2,11,9,0,0,0,1,1899,1701,8,17,3,198,1
4,2000_1_Oakland Raiders_San Diego Chargers,Las Vegas Raiders,Los Angeles Chargers,2000-09-03,2000,9,1,6,0,9,6,20,19,35,42,166,179,1,0,0,3,4,2,10,18,5.0,4.7,4.3,4.1,57.1,45.2,79.0,27.8,25,28,67,76,2.7,2.7,0,1,0,0,1,0,1,0,1,0,10,7,420,343,3,8,16,18,0,0,0,1,1890,1710,3,-13,-9,180,1


In [25]:
df.loc[5647, ['home_pass_yds']]

home_pass_yds    177
Name: 5647, dtype: object

In [ ]:
sddsdddddd

In [35]:
df['home_total_yds']= df['home_pass_yds'] + df['home_rush_yds']
df['away_total_yds']= df['away_pass_yds'] + df['away_rush_yds']

In [37]:
df['home_results'] = df.pop('home_results')

In [39]:
df.loc[5647]

game_id               2022_1_Seattle Seahawks_Denver Broncos
home_team                                   Seattle Seahawks
away_team                                     Denver Broncos
date                                     2022-09-12 00:00:00
season                                                  2022
                                       ...                  
rush_yds_diff_home                                       -27
top_diff_home                                           -438
home_total_yds                                           253
away_total_yds                                           433
home_results                                               1
Name: 5647, Length: 70, dtype: object

Data Dictionary:

game_id - a unique identifier for each game played with the season, week, home team, and away team
home_team - the home team for that game
away_team - the away team fot that game
date - date of the game played on 
season - the year the season was played
month - refers to the month the game played in (i.e. 10 = October, 12 = December)
week - refers to the week of the NFL season 
day_of_week - refers to the day the game was played on (i.e. 6 = Sunday, 0 = Monday)
overtime - 0 refers to no overtime and 1 refers to yes there was overtime in the game
home_pts - refers to the amount of points the home team scored
away_pts - refers to the amount of points the awat team scored 
home_pass_cmp - home team passing completions
away_pass_cmp - away team passing completions
home_pass_att - home team passing attempts
away_pass_att - away team passing attempts
home_pass_yds - home team passing yards
away_pass_yds - away team passing yards
home_pass_td - home team passing touchdowns
away_pass_td - away team passing touchdowns
home_int - home team interceptions thrown
away_int - away team interceptions thrown
home_sk - home team allowed sacks
away_sk - away team allowed sacks
home_yds_lost_sks - home team lost yards from sacks
away_yds_lost_sks - away team lost yards from sacks
home_yds_pass_att - home team yards per pass attempt
away_yds_pass_att - away team yards per pass attempt
home_net_yds_pass_att - home team net yards per pass
away_net_yds_pass_att - away team net yards per pass
home_cmp% - home team passing completion percent
away_cmp% - away team passing completion percent
home_pass_rating - home team passer rating
away_pass_rating - away team passer rating
home_rush_att - home team rushing attempts
away_rush_att - away team rushing attempts
home_rush_yds - home team rushing yards
away_rush_yds - away team rushing yards
home_rush_yds_att - home team rushing yards per attempt
away_rush_yds_att - away team rushing yards per attempt
home_rush_td - home team rushing touchdowns
away_rush_td - away teams rushing touchdowns
home_fgm - home team field goals made
away_fgm - away team field goals made
home_fga - home team field goals attempts
away_fga - away team field goals attempts
home_xpm - home team extra points made
away_xpm - away team extra points made
home_xpa - home team extra points attempts
away_xpa - away team extra points attempts
home_pnt - home team number of punts 
away_pnt - away team number of punts
home_punt_yds - home team punt yards
away_punt_yds - away team punt yards
home_3dconv - home team 3rd down conversions 
away_3dconv - away team 3rd down conversions
home_3datt - home team 3rd down attempts
away_3datt - away team 3rd down attempts
home_4dconv - home team 4th down conversions
away_4dconv - away team 4th down conversions
home_4datt - home team 4th down attempts
away_4datt - away team 4th down attempts 
home_top - home team time of possession
away_top - away team time of possession 
pts_diff_home - points differential in perspective of the home team (i.e. -5 means the away team scored 5 more points than the home team, 7 means the home team scored 7 more points than the away team)

pass_yds_diff_home - passing yards differential in perspective of the home team (i.e. negative passing yards means the away team had that number more of the home team, positive yards means the home team had that many more passing yards than the away team)

rush_yds_diff_home - rushing yards differential in the perspective of the home team(i.e. (i.e. negative rushing yards means the away team had that number more of the home team, positive yards means the home team had that many more rushing yards than the away team)

top_diff_home - time of possession differential in the perspective of the home team(i.e. negative time means the away team had that more of time of possession than the home team, if the time is positive that means the home team had that amount of time of possession more than the away team.) Also, time is recorded in seconds 

home_total_yds - home team total passing and rushing yards

away_total_yds - away team total passing and rushing yards

home_results - 0 refers to the away team winning and 1 refers to the home team winning

In [40]:
df.to_csv('getting_there.csv', index = False)